In [1]:
from pyspark.sql import functions as F
from pyspark.sql.types import *

In [2]:
list=[[1,12.1231231245,'20200102',1,2,3,4,5]]

df=spark.createDataFrame(list,['code','part_num','start_date','daily_qty_1','daily_qty_2','daily_qty_3','daily_qty_4','daily_qty_5'])
df.show()

+----+--------+----------+-----------+-----------+-----------+-----------+-----------+
code|part_num|start_date|daily_qty_1|daily_qty_2|daily_qty_3|daily_qty_4|daily_qty_5|
+----+--------+----------+-----------+-----------+-----------+-----------+-----------+
 1| 12| 20200102| 1| 2| 3| 4| 5|
+----+--------+----------+-----------+-----------+-----------+-----------+-----------+

In [3]:
from pyspark.sql import functions as F
df.withColumn("daily_qty", F.split(F.concat_ws(',',*(x for x in df.columns if x.startswith('daily_qty'))),','))\
.withColumn("size", F.size("daily_qty"))\
.withColumn("start_date",F.col("start_date").cast("long"))\
.withColumn("date", F.expr("""sequence(start_date,start_date+bigint(size)-1,1)""")).drop("size")\
.withColumn("temp", F.explode(F.arrays_zip("date", "daily_qty")))\
.select("code","part_num","temp.date","temp.daily_qty")\
.withColumn("date", F.to_date(F.col("date").cast("string"),"yyyyMMdd")).show()

+----+--------+----------+---------+
code|part_num| date|daily_qty|
+----+--------+----------+---------+
 1| 12|2020-01-02| 1|
 1| 12|2020-01-03| 2|
 1| 12|2020-01-04| 3|
 1| 12|2020-01-05| 4|
 1| 12|2020-01-06| 5|
+----+--------+----------+---------+

+----+--------+----------+---------+
code|part_num| date|daily_qty|
+----+--------+----------+---------+
 1| 12|2020-01-02| 1|
 1| 12|2020-01-03| 2|
 1| 12|2020-01-04| 3|
 1| 12|2020-01-05| 4|
 1| 12|2020-01-06| 5|
+----+--------+----------+---------+

In [5]:
x='1234.345.344343434'
x[x.rfind(".") + 1:]

Out[35]: '344343434'

In [6]:
from pyspark.sql import functions as F
list=[['1234.345.344343434']]
df=spark.createDataFrame(list,['value'])
df.show()
df.withColumn("value", F.element_at(F.split(F.regexp_replace("value","\.",','),','),-1)).show()

+------------------+
 value|
+------------------+
1234.345.344343434|
+------------------+

+---------+
 value|
+---------+
344343434|
+---------+

In [7]:
from pyspark.sql import functions as F
list=[['1234.345.344343434']]
df=spark.createDataFrame(list,['value'])
df.show()

+------------------+
 value|
+------------------+
1234.345.344343434|
+------------------+

In [8]:
df.withColumn("value", F.regexp_replace("value", '\.',',')).show()

+------------------+
 value|
+------------------+
1234,345,344343434|
+------------------+

In [9]:
df.show()

+---+--------------+--------+
one| two| three|
+---+--------------+--------+
 1|223.0000000000|20200102|
+---+--------------+--------+

In [10]:
df.withColumn("value", F.element_at(F.split("value",'\.'),-1)).show()

+---------+
 value|
+---------+
344343434|
+---------+

In [11]:
order_udf = F.udf(lambda x: x[x.rfind(".") + 1:], StringType())
df.withColumn("order_number", order_udf('value')).show()

+------------------+------------+
 value|order_number|
+------------------+------------+
1234.345.344343434| 344343434|
+------------------+------------+

In [12]:
from pyspark.sql import functions as F
from pyspark.sql.types import *

list=[[1,223,'20200102']]
cschema= StructType([StructField('one', IntegerType()),
                    StructField('two', LongType()),
                    StructField('three', StringType())])
df=spark.createDataFrame(list,schema=cschema)

df=df.withColumn("two", F.col("two").cast(DecimalType(38,10)))

In [13]:
df.printSchema()

root
-- one: integer (nullable = true)
-- two: decimal(38,10) (nullable = true)
-- three: string (nullable = true)

In [14]:
def dropdcols(df):
    l=[]
    for i in range(len(df.columns)):
       if df.dtypes[i][1] =='decimal(38,10)':
          l.append(df.columns[i])
    
    return df.drop(*l)
          
          

  
  
dropstrcols(df).show()

+---+--------+
one| three|
+---+--------+
 1|20200102|
+---+--------+

In [15]:
df1.dtypes

Out[18]: [('one', 'int'), ('two', 'decimal(38,10)'), ('three', 'string')]

In [16]:
list=[['8A',1,'CPH','GDN'],                
      ['8A',1,'GDN','CPH'],                 
      ['8A',2,'GDN','CPH'],               
      ['8A',2,'CPH','GDN'],                
      ['8A',3,'CPH','GDN'],                 
      ['8A',3,'GDN','CPH'],                 
      ['8A',4,'CPH','GDN'],                  
      ['8A',4,'GDN','CPH'],
      ['8A',5,'GDN','GDN'],
      ['8A',5,'CPH','GDN']] 
df=spark.createDataFrame(list,['id','val_no','stn_fr_cd','stn_to_cd'])
df.show()

+---+------+---------+---------+
 id|val_no|stn_fr_cd|stn_to_cd|
+---+------+---------+---------+
 8A| 1| CPH| GDN|
 8A| 1| GDN| CPH|
 8A| 2| GDN| CPH|
 8A| 2| CPH| GDN|
 8A| 3| CPH| GDN|
 8A| 3| GDN| CPH|
 8A| 4| CPH| GDN|
 8A| 4| GDN| CPH|
 8A| 5| GDN| GDN|
 8A| 5| CPH| GDN|
+---+------+---------+---------+

In [17]:
from pyspark.sql import functions as F
from pyspark.sql.window import Window
w=Window().partitionBy("id","val_no").orderBy("val_no")
w2=Window().partitionBy("id")

df.withColumn("fr", F.lead("stn_fr_cd").over(w))\
  .withColumn("to", F.lead("stn_to_cd").over(w))\
  .withColumn("check", F.when((F.col("stn_fr_cd")==F.col("to"))&(F.col("stn_to_cd")==F.col("fr")),F.lit(1)).otherwise(F.lit(0)))\
  .groupBy("id").agg(F.sum("check").alias("diagonals")).show()

+---+---------+
 id|diagonals|
+---+---------+
 8A| 4|
+---+---------+

In [18]:
from pyspark.sql import functions as F

In [19]:
cols=['USA','CAN','UK','DEN']
nums=[1,2]

list=[[98,1,0,1,1],
      [99,0,1,0,0]]

df=spark.createDataFrame(list, ['ID','1_USA','1_DEN','2_DEN','1_UK'])
df.show()

+---+-----+-----+-----+----+
 ID|1_USA|1_DEN|2_DEN|1_UK|
+---+-----+-----+-----+----+
 98| 1| 0| 1| 1|
 99| 0| 1| 0| 0|
+---+-----+-----+-----+----+

In [20]:
l=[]
for i in cols:
  for j in nums:
    l.append(str(j)+'_'+i)
    
for i in l:
     if i not in df.columns:
        df1=df.withColumn(i,F.lit("0"))
        
df1.show()

+---+-----+-----+-----+----+----+
 ID|1_USA|1_DEN|2_DEN|1_UK|2_UK|
+---+-----+-----+-----+----+----+
 98| 1| 0| 1| 1| 0|
 99| 0| 1| 0| 0| 0|
+---+-----+-----+-----+----+----+

In [21]:
l

Out[5]: ['1_USA', '2_USA', '1_CAN', '2_CAN', '1_UK', '2_UK', '1_DEN', '2_DEN']

In [22]:
+---+------+---------+---------+----+----+-----+
| id|val_no|stn_fr_cd|stn_to_cd|  fr|  to|check|
+---+------+---------+---------+----+----+-----+
| 8A|     2|      GDN|      CPH| CPH| GDN|    1|
| 8A|     2|      CPH|      GDN|null|null|    0|
| 8A|     3|      CPH|      GDN| GDN| CPH|    1|
| 8A|     3|      GDN|      CPH|null|null|    0|
| 8A|     4|      CPH|      GDN| GDN| CPH|    1|
| 8A|     4|      GDN|      CPH|null|null|    0|
| 8A|     1|      CPH|      GDN| GDN| CPH|    1|
| 8A|     1|      GDN|      CPH|null|null|    0|
| 8A|     5|      GDN|      GDN| CPH| GDN|    0|
| 8A|     5|      CPH|      GDN|null|null|    0|
+---+------+---------+---------+----+----+-----+

In [23]:
data=  [[100,1],
        [101,1],  
        [102,1],
        [103,2],
        [104,2],
        [105,12],
        [106,12],
        [107,1],
        [108,2],
        [109,12],
        [110,2],
        [111,0],
        [112,22],
        [113,17],
        [114,20]]
columns= ['SecondsInHour','Total']
df= spark.createDataFrame(data,columns)
df.show()

+-------------+-----+
SecondsInHour|Total|
+-------------+-----+
 100| 1|
 101| 1|
 102| 1|
 103| 2|
 104| 2|
 105| 12|
 106| 12|
 107| 1|
 108| 2|
 109| 12|
 110| 2|
 111| 0|
 112| 22|
 113| 17|
 114| 20|
+-------------+-----+

In [24]:
from pyspark.sql import functions as F
from pyspark.sql.window import Window
w2= Window().orderBy(df.SecondsInHour)
df.withColumn("lagdiff",(F.col("Total")- F.lag("Total").over(w2)))\
.withColumn("lagdiff3", F.when((F.col("lagdiff")>0),F.lit(0)).otherwise(F.col("lagdiff")*-1))\
.withColumn("lagdiff4", F.when((F.col("lagdiff") <0), F.lit(0)).otherwise(F.col("lagdiff")))\
.withColumn("In", F.when(F.col("lagdiff").isNull(), F.col("Total")).otherwise(F.col("lagdiff4")))\
.withColumn("Out",F.when(F.col("lagdiff3").isNull(), F.lit(0)).otherwise(F.col("lagdiff3")))\
.orderBy(df.SecondsInHour.asc())\
.show()

+-------------+-----+-------+--------+--------+---+---+
SecondsInHour|Total|lagdiff|lagdiff3|lagdiff4| In|Out|
+-------------+-----+-------+--------+--------+---+---+
 100| 1| null| null| null| 1| 0|
 101| 1| 0| 0| 0| 0| 0|
 102| 1| 0| 0| 0| 0| 0|
 103| 2| 1| 0| 1| 1| 0|
 104| 2| 0| 0| 0| 0| 0|
 105| 12| 10| 0| 10| 10| 0|
 106| 12| 0| 0| 0| 0| 0|
 107| 1| -11| 11| 0| 0| 11|
 108| 2| 1| 0| 1| 1| 0|
 109| 12| 10| 0| 10| 10| 0|
 110| 2| -10| 10| 0| 0| 10|
 111| 0| -2| 2| 0| 0| 2|
 112| 22| 22| 0| 22| 22| 0|
 113| 17| -5| 5| 0| 0| 5|
 114| 20| 3| 0| 3| 3| 0|
+-------------+-----+-------+--------+--------+---+---+